In [102]:
# Our 3 layer NN MNIST in numpy from scratch
import sys, numpy as np
from keras.datasets import mnist
np.random.seed(1)


(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


In [105]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [108]:
images, labels = (x_train[0:1000].reshape(1000,28*28)/255, y_train[0:1000])

In [109]:
images.shape

(1000, 784)

In [112]:
one_hot_labels = np.zeros((len(labels), 10))

In [113]:
one_hot_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [141]:
labels[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [116]:
for i, l in enumerate(labels):
    one_hot_labels[i][l] = 1

In [117]:
labels = one_hot_labels

In [120]:
labels[1]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [125]:
# check X_test images

test_images = x_test.reshape((len(x_test), 28 * 28))
test_labels  = np.zeros((len(y_test), 10))


In [126]:
test_images.shape

(10000, 784)

In [130]:
test_labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [131]:
for i, l in enumerate(y_test):
    test_labels[i][l] = 1

In [132]:
test_labels[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [154]:
relu = lambda x:(x>=0) * x
relu2deriv = lambda x: x>=0
alpha, iterations, hidden_size, pixels_per_image, num_labels = \
(0.005, 350, 40, 784, 10)
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

In [143]:
weights_0_1.shape, weights_1_2.shape

((784, 40), (40, 10))

In [144]:
labels[0:1].shape

(1, 10)

### Variation 1 - As per the book

In [164]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0.0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        #print("error==>>", error)
        correct_cnt += int(np.argmax(layer_2) == \
        np.argmax(labels[i:i+1]))
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)\
        * relu2deriv(layer_1)
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
        
    sys.stdout.write("\r"+ \
    " I:"+str(j)+ \
    " Error:" + str(error/float(len(images)))[0:5] +\
    " Correct:" + str(correct_cnt/float(len(images))))
            
        
        

 I:349 Error:0.086 Correct:0.999

### Variation 2 - Substracting weight deltas 

In [191]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0.0)
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        #print("Error", error)
        correct_cnt += int(np.argmax(layer_2) == \
        np.argmax(labels[i:i+1]))
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)\
        * relu2deriv(layer_1)
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 -= alpha * layer_0.T.dot(layer_1_delta)
    sys.stdout.write("\r"+ \
    " I:"+str(j)+ \
    " Error:" + str(error/float(len(images)))[0:5] +\
    " Correct:" + str(correct_cnt/float(len(images))))

/var/folders/q2/8nypxvkj1xvdpkrrjnk711rw0000gn/T/ipykernel_57826/508043021.py:14: RuntimeWarning: overflow encountered in square
  error += np.sum((labels[i:i+1] - layer_2) ** 2)


 I:349 Error:nan Correct:0.097

In [162]:
if(j % 10 == 0 or j == iterations-1):
    error, correct_cnt = (0.0, 0)
for i in range(len(test_images)):
    layer_0 = test_images[i:i+1]
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == \
    np.argmax(test_labels[i:i+1]))
sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] +\
" Test-Acc:" + str(correct_cnt/float(len(test_images))))
print()

 Test-Err:71531 Test-Acc:0.6882
